In [3]:
import pandas as pd

In [2]:
corpus=pd.read_csv('corpus.csv')
corpus.head()

,text,cid
0,"Thông tư này hướng dẫn tuần tra, canh gác bảo ...",0
1,"1. Hàng năm trước mùa mưa, lũ, Ủy ban nhân dân...",1
2,Tiêu chuẩn của các thành viên thuộc lực lượng ...,2
3,"Nhiệm vụ của lực lượng tuần tra, canh gác đê\n...",3
4,"Phù hiệu của lực lượng tuần tra, canh gác đê\n...",4


In [6]:
def get_texts_by_cids(df, cids):
    """
    Retrieve texts corresponding to the given list of cids from the DataFrame.

    Args:
    df (pd.DataFrame): DataFrame containing 'text' and 'cid' columns.
    cids (list): List of cids to search for.

    Returns:
    list: List of texts corresponding to the given cids.
    """
    # Filter the DataFrame based on the cids
    texts = df[df['cid'].isin(cids)]['text'].tolist()
    return texts
print(get_texts_by_cids(corpus,[1,2,3]))

['1. Hàng năm trước mùa mưa, lũ, Ủy ban nhân dân cấp xã nơi có đê phải tổ chức lực lượng lao động tại địa phương để tuần tra, canh gác đê và thường trực trên các điếm canh đê hoặc nhà dân khu vực gần đê (đối với những khu vực chưa có điếm canh đê), khi có báo động lũ từ cấp I trở lên đối với tuyến sông có đê (sau đây gọi tắt là lực lượng tuần tra, canh gác đê).\n2. Lực lượng tuần tra, canh gác đê được tổ chức thành các đội, do Ủy ban nhân dân cấp xã ra quyết định thành lập; từ 01 đến 02 kilômét đê thành lập 01 đội; mỗi đội có từ 12 đến 18 người, trong đó có 01 đội trưởng và 01 hoặc 02 đội phó. Danh sách thành viên đội tuần tra, canh gác đê được niêm yết tại điếm canh đê thuộc địa bàn được phân công.\n3. Khi lũ, bão có diễn biến phức tạp, kéo dài ngày, Uỷ ban nhân dân cấp xã có thể quyết định việc bổ sung thêm thành viên cho đội tuần tra, canh gác đê.', 'Tiêu chuẩn của các thành viên thuộc lực lượng tuần tra, canh gác đê\n1. Là người khoẻ mạnh, tháo vát, đủ khả năng đảm đương những công

In [3]:
cids=corpus['cid'].to_list()
print(cids[0])

0


In [95]:
train=pd.read_csv('train_with_hard_negative.csv')
train.head()

,question,context,cid,qid,hard_negative
0,Người học ngành quản lý khai thác công trình t...,"['Khả năng học tập, nâng cao trình độ\n- Khối ...",[62492],161615,"[161615, 189736, 574340, 227186, 498944, 14382..."
1,Nội dung lồng ghép vấn đề bình đẳng giới trong...,['Nội dung lồng ghép vấn đề bình đẳng giới tro...,[151154],80037,"[80037, 53914, 214112, 26107, 151153, 26117, 2..."
2,Sản phẩm phần mềm có được hưởng ưu đãi về thời...,"['""Điều 20. Ưu đãi về thời gian miễn thuế, giả...",[75071],124074,"[124074, 585406, 233023, 631699, 629619, 45449..."
3,Điều kiện để giáo viên trong cơ sở giáo dục mầ...,"['Điều kiện được hưởng\nCán bộ quản lý, giáo v...",[225897],146841,"[146841, 259260, 603297, 11935, 552192, 603295..."
4,Nguyên tắc áp dụng phụ cấp ưu đãi nghề y tế th...,"['Nguyên tắc áp dụng\n1. Trường hợp công chức,...",[68365],6176,"[6176, 111914, 49288, 92768, 81573, 71511, 492..."


In [96]:
length={}
data_records=train.to_dict(orient='records')
for point in data_records:
    cid_str=point['cid']
    curr=list(map(int, cid_str.strip('[]').split()))
    curr_text=point['context']
    if len(curr) != len(curr_text.split('\n')):
        break
    if len(curr) in length.keys():
        length[len(curr)]+=1
    else:
        length[len(curr)]=1
print(length)

{1: 106884, 2: 11282, 3: 1115, 4: 128, 5: 30, 7: 3, 6: 11, 9: 2, 8: 1}


In [97]:
def process_context(context : str):
    context_list=context.split('\n')
    context_strip=[law.strip('[]').replace('"', '').replace("'","") for law in context_list]
    return context_strip
train['context']=train['context'].apply(process_context)
train['cid']=train['cid'].apply(lambda x:list(map(int, x.strip('[]').split())))
train['hard_negative']=train['hard_negative'].apply(eval)

In [98]:

# Function to expand the context column into separate positive columns
def expand_context_column(df : pd.DataFrame, context_col='context'):
    df=df.copy()
    # Get the maximum length of lists in the context column
    max_length = df[context_col].apply(len).max()
    if max_length>10:
        return 0
    print('max length of positive',max_length)
    # Create new columns for each positive text
    for i in range(max_length):
        # Create a new column name
        new_col_name = f'positive_{i + 1}'
        
        # Assign the i-th element of the lists, if it exists; otherwise, assign NaN
        df[new_col_name] = df[context_col].apply(lambda x: x[i] if i < len(x) else None)
    removed_df = df.drop(columns=[context_col])
    return removed_df

# Expand the context column
train_2 = expand_context_column(train,'context')
train_2.head()


max length of positive 9


,question,cid,qid,hard_negative,positive_1,positive_2,positive_3,positive_4,positive_5,positive_6,positive_7,positive_8,positive_9
0,Người học ngành quản lý khai thác công trình t...,[62492],161615,"[161615, 189736, 574340, 227186, 498944, 14382...","Khả năng học tập, nâng cao trình độ\n- Khối lư...",None,None,None,None,None,None,None,None
1,Nội dung lồng ghép vấn đề bình đẳng giới trong...,[151154],80037,"[80037, 53914, 214112, 26107, 151153, 26117, 2...",Nội dung lồng ghép vấn đề bình đẳng giới trong...,None,None,None,None,None,None,None,None
2,Sản phẩm phần mềm có được hưởng ưu đãi về thời...,[75071],124074,"[124074, 585406, 233023, 631699, 629619, 45449...","Điều 20. Ưu đãi về thời gian miễn thuế, giảm t...",None,None,None,None,None,None,None,None
3,Điều kiện để giáo viên trong cơ sở giáo dục mầ...,[225897],146841,"[146841, 259260, 603297, 11935, 552192, 603295...","Điều kiện được hưởng\nCán bộ quản lý, giáo viê...",None,None,None,None,None,None,None,None
4,Nguyên tắc áp dụng phụ cấp ưu đãi nghề y tế th...,[68365],6176,"[6176, 111914, 49288, 92768, 81573, 71511, 492...","Nguyên tắc áp dụng\n1. Trường hợp công chức, v...",None,None,None,None,None,None,None,None


In [101]:
def expand_hn_column(df : pd.DataFrame, context_col='hard_negative'):
    df=df.copy()
    # Get the maximum length of lists in the context column
    max_length = df[context_col].apply(len).max()
    if max_length>15:
        return 0
    print('max length of positive',max_length)
    # Create new columns for each positive text
    for i in range(max_length):
        # Create a new column name
        new_col_name = f'negative_{i + 1}'
        
        # Assign the i-th element of the lists, if it exists; otherwise, assign NaN
        df[new_col_name] = df[context_col].apply(lambda x: x[i] if i < len(x) else None)
    removed_df = df.drop(columns=[context_col])
    return removed_df

[161615, 189736, 574340, 227186, 498944, 143824, 61726, 131679, 494399, 77623]